# Airline AI Agent
## Image creation using tools

In week2 day5's project I realized that the asisstant would always render an image of the second city provided on the compare two city prices query instead of rendering the image of the cheaper one.

I though that perhaps this might be solved by using tools to rendering the images rather than a plain function call in the chat method.

Since it sounded like a good tools exercise, I decided to jump into it and created the modified code below.

It works fine from a tools perspective but unfortunatelly when I ask the agent about a cheaper option between two cities it does not render any image at all even though instructed to do so in the system prompt.

Could you please provide some insight about what might be going on and suggestions about how to better promt it?

Thanks

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

import base64
from io import BytesIO
from PIL import Image

import re


In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

#MODEL = "gpt-4o-mini"

MODEL = "gpt-5-mini"

#MODEL = "gpt-oss:20b"

#OLLAMA_BASE_URL = "http://localhost:11434/v1"

#ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
openai = OpenAI()

DB = "prices.db"

In [ ]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
if your answer has one or more cities always create a picture of the most relevant and include the name of the file provided by the tool
"""
# Always create a picture of the most relevant city on your answer and include the name of the file provided by the tool
# if needed do multiple tool requests before rendering a final answer

In [ ]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [ ]:
def get_image(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return image_data
    #return Image.open(BytesIO(image_data))

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="alloy",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content   

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
image_function = {
    "name": "get_image",
    "description": "Get the image of a city",
    "parameters": {
        "type": "object",
        "properties": {"city": {"type": "string", "description": "The city that an image of is required"}},
        "required": ["city"],
        "additionalProperties": False
    }
}

In [ ]:
# talker_function = {
#     "name": "talker",
#     "description": "Talk to the customer",
#     "parameters": {
#         "type": "object",
#         "properties": {"message": {"type": "string", "description": "The message to talk to the customer"}},
#         "required": ["message"],
#         "additionalProperties": False
#     }
# }


In [ ]:
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": image_function}] #, {"type": "function", "function": talker_function}]

In [ ]:
tools

In [ ]:
def extract_image_filename(response_text):
    pattern = r"\b[\w\-.]+\.(png|jpg|jpeg|gif|bmp|webp)\b"
    match = re.search(pattern, response_text, flags=re.IGNORECASE)
    if match:
        imageFile = match.group(0)
        return imageFile
    return None

# def extract_audio_filename(response_text):
#    pattern = r"\b[\w\-.]+\.(mp3|wav|flac|aac|ogg|m4a)\b"
#    match = re.search(pattern, response_text, flags=re.IGNORECASE)
#    if match:
#        audioFile = match.group(0)
#        return audioFile
#    return None

def load_image_bytes(path):
    with open(path, "rb") as f:
        return f.read()

# def load_audio(path):
#    with open(path, "rb") as f:
#        return f.read()


In [ ]:
def chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    #cities = []
    image = None
    imageFile = None
    audioFile = None
    voice = None

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)      

        for message in messages:
            print(message)
            
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = talker(reply)

    print(reply)

    # if cities:
    #     image = get_image(cities[0])

    imageFile = extract_image_filename(reply)
    if imageFile:
        image_bytes = load_image_bytes(imageFile)
        image = Image.open(BytesIO(image_bytes))
    
    #audioFile = extract_audio_filename(reply)
    #voice = load_audio(audioFile)
    
    return history, voice, image

In [ ]:
def handle_tool_calls(message):
    responses = []
    #cities = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            #cities.append(city)
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "get_image":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('city')
            #cities.append(city)
            image = get_image(city)
            with open(f"image{city}.png", "wb") as f:
                f.write(image)
            responses.append({
                "role": "tool",
                "content": f"image succesfully created in : image{city}.png, make sure to include this file name on any responce referencing this image",
                "tool_call_id": tool_call.id
            })
        # elif tool_call.function.name == "talker":
        #     arguments = json.loads(tool_call.function.arguments)
        #     message = arguments.get('message')
        #     audio = talker(message)
        #     #with open("audioFile.mp3", "wb") as f:
        #     #    f.write(audio)
        #     responses.append({
        #         "role": "tool",
        #         "content": "audio succesfully created in : audioFile.mp3",
        #         "tool_call_id": tool_call.id
        #     })
        
    return responses

In [ ]:
# Callbacks (along with the chat() function above)

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )


In [ ]:
#force_dark_mode = """
#function refresh() {
#    const url = new URL(window.location);
#    if (url.searchParams.get('__theme') !== 'dark') {
#        url.searchParams.set('__theme', 'dark');
#        window.location.href = url.href;
#    }
#}
#"""    

#gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()
ui.launch(inbrowser=True) #, auth=("francisco", "tools"))